In [29]:
%run partialOrder.ipynb

..........................................................................E.....................................
ERROR: test_explanator_01 (__main__.TestNotebook_explanator)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-25-657e1747ceca>", line 26, in test_explanator_01
    for p in generator:
  File "<ipython-input-23-d8ead54934aa>", line 376, in analyze_all
    yield from self.analyze(condition,0,True)
  File "<ipython-input-23-d8ead54934aa>", line 343, in analyze
    yield from self.analyze(node.children[1], level+1, True)
  File "<ipython-input-23-d8ead54934aa>", line 354, in analyze
    if interpreter.interpret(node.children[0]) == False:
NameError: name 'interpreter' is not defined

----------------------------------------------------------------------
Ran 112 tests in 1.724s

FAILED (errors=1)
..........................................................................E...............................

In [40]:
from lark import Lark, Tree, Token

class Interpreter:
    
    def is_valid(self, relation, universe):
        if not (relation.get_domain() | relation.get_codomain()) <= FiniteSet(*universe):
            raise Exception(f"Relation - {relation} not valid for universe - {universe}!")
    
    def __init__(self, variables=dict(), universe=FiniteSet(), check=False):
        if check == True:
            for value in variables.values():
                self.is_valid(value, universe)
                
        self._variables = variables
        self._universe = FiniteSet(*universe)
        self._parser = Lark('''
                        RELATION: "P".."Z"
                        SET: "A".."O"
                        ?logical_operator: special_operator
                             | logical_operator "and" special_operator  -> conjunction
                             | logical_operator "or"  special_operator  -> disjunction                   
                             | logical_operator "->"  special_operator  -> implication
                             | logical_operator "=="  special_operator  -> equal
                             | logical_operator "!="  special_operator  -> not_equal
                             | logical_operator ">"   special_operator  -> greater_than
                             | logical_operator "<"   special_operator  -> lower_than
                             | logical_operator ">="  special_operator  -> greater_equal
                             | logical_operator "<="  special_operator  -> lower_equal                   
                        ?special_operator: properties
                             | "not" logical_operator      -> negation
                             | product "is" properties     -> is
                             | product "is not" properties -> is_not
                        ?properties: product    
                             | "total_order"   -> total_order
                             | "transitive"    -> transitive
                             | "lattice"       -> lattice
                             | "reflexive"     -> reflexive
                             | "symmetric"     -> symmetric
                             | "asymmetric"    -> asymmetric
                             | "antisymmetric" -> antisymmetric
                        ?product: atom
                             | product "°" atom   -> composition
                             | product "&" atom   -> intersection
                             | product "|" atom   -> union
                             | product "-" atom   -> difference
                             | product "^" atom   -> symmetric_difference
                             | product "*" atom   -> multiplication
                        ?atom: "("logical_operator")"
                             | SET 
                             | RELATION
                        %import common.WS
                        %ignore WS
                 ''', start='logical_operator')
        
    def parse(self, statement):
        return self._parser.parse(statement)
        
    def set_variables(self, variables=dict(), check=False):
        if check == True:
            for value in variables.values():
                self.is_valid(value, self.get_universe())
        self._variables = variables
        
    def get_variables(self, *variables):
        if len(variables) == 0:
            return self._variables
        else:
            selected_variables = dict()
            for variable in variables:
                selected_variables[variable] = self._variables[variable]
            return selected_variables
    
    def add_variable(self, variable, value, check=False):
        if check == True:
            self.is_valid(value, self.get_universe())
        variables = self.get_variables()
        if variable in variables.keys():
            raise Exception(f"Variable - '{variable}' is already defined!")
        variables[variable] = value
        self.set_variables(variables)
        
    def set_variable(self, variable, new_value, check=False):
        if check == True:
            self.is_valid(new_value, self.get_universe())
        variables = self.get_variables()
        if variable not in variables.keys():
            raise Exception(f"Variable - '{variable}' has not been defined yet!")
        variables[variable] = new_value
        self.set_variables(variables)
        
    def set_universe(self, universe=FiniteSet(), check=False):
        if check == True:
            for relation in self.get_variables().values():
                if not (relation.get_domain() | relation.get_codomain()) <= FiniteSet(*universe):
                    raise Exception(f"Universe - {universe} not valid for relation - {relation}!") 
        self._universe = FiniteSet(*universe)
        
    def get_universe(self):
        return self._universe
        
    def constant(self,data):
        try:
            variables = self.get_variables()
            return variables[data]
        except:
            raise Exception(f"Variable - '{data}' doesn't exist!")

    def interpret(self,node):
        if isinstance(node, Tree):
            if node.data == 'negation':
                return not self.interpret(node.children[0])
            elif len(node.children) == 2:
                left, right = node.children  
                if node.data == 'equal':
                    return self.interpret(left) == self.interpret(right)
                elif node.data == 'not_equal':
                    return self.interpret(left) != self.interpret(right)
                elif node.data == 'greater_than':
                    return self.interpret(left) > self.interpret(right)
                elif node.data == 'lower_than':
                    return self.interpret(left) < self.interpret(right)
                elif node.data == 'greater_equal':
                    return self.interpret(left) >= self.interpret(right)
                elif node.data == 'lower_equal':
                    return self.interpret(left) <= self.interpret(right)
                elif node.data == 'conjunction':
                    return self.interpret(left) and self.interpret(right)
                elif node.data == 'disjunction':
                    return self.interpret(left) or self.interpret(right)
                elif node.data == 'implication':
                    return (not self.interpret(left)) or self.interpret(right)
                elif node.data == 'intersection':
                    return self.interpret(left) & self.interpret(right)
                elif node.data == 'union':
                    return self.interpret(left) | self.interpret(right)
                elif node.data == 'difference':
                    return self.interpret(left) - self.interpret(right)
                elif node.data == 'symmetric_difference':
                    return self.interpret(left) ^ self.interpret(right)
                elif node.data == 'multiplication':
                    return self.interpret(left) * self.interpret(right)
                elif node.data == 'composition':
                    return self.interpret(left).composition(self.interpret(right))
                elif node.data == 'is' or node.data == 'is_not':
                    if right.data == 'lattice':
                        return self.interpret(left).is_lattice() if node.data == 'is' else not self.interpret(left).is_lattice()
                    elif right.data == 'total_order':
                        return self.interpret(left).is_total_order() if node.data == 'is' else not self.interpret(left).is_total_order()
                    elif right.data == 'reflexive':
                        return self.interpret(left).is_reflexive() if node.data == 'is' else not self.interpret(left).is_reflexive()
                    elif right.data == 'symmetric':
                        return self.interpret(left).is_symmetric() if node.data == 'is' else not self.interpret(left).is_symmetric()
                    elif right.data == 'asymmetric':
                        return self.interpret(left).is_asymmetric() if node.data == 'is' else not self.interpret(left).is_asymmetric()
                    elif right.data == 'antisymmetric':
                        return self.interpret(left).is_antisymmetric() if node.data == 'is' else not self.interpret(left).is_antisymmetric()
                    elif right.data == 'transitive':
                        return self.interpret(left).is_transitive() if node.data == 'is' else not self.interpret(left).is_transitive()
                else:
                    raise Exception(f"ERROR! {node.data}")
            else:
                pass
                #raise Exception(f"ERROR! Unexpected input! {node}")
        elif isinstance(node, Token):
            node = self.constant(node.value)
            return node
        else:
            raise Exception(f"ERROR! unkonown instance {type(node)}")
            
    def node2string(self,node):
        if isinstance(node, Tree):
            if node.data == 'negation':
                return "not " + self.node2string(node.children[0])
            elif len(node.children) == 2:
                left, right = node.children  
                if node.data == 'equal':
                    return "(" + self.node2string(left) + " == " + self.node2string(right) + ")"
                elif node.data == 'not_equal':
                    return "(" + self.node2string(left) + " != " + self.node2string(right) + ")"
                elif node.data == 'greater_than':
                    return "(" + self.node2string(left) + " > " + self.node2string(right) + ")"
                elif node.data == 'lower_than':
                    return "(" + self.node2string(left) + " < " + self.node2string(right) + ")"
                elif node.data == 'greater_equal':
                    return "(" + self.node2string(left) + " >= " + self.node2string(right) + ")"
                elif node.data == 'lower_equal':
                    return "(" + self.node2string(left) + " <= " + self.node2string(right) + ")"
                elif node.data == 'conjunction':
                    return "(" + self.node2string(left) + " and " + self.node2string(right) + ")"
                elif node.data == 'disjunction':
                    return "(" + self.node2string(left) + " or " + self.node2string(right) + ")"
                elif node.data == 'implication':
                    return "(" + self.node2string(left) + " -> " + self.node2string(right) + ")"
                elif node.data == 'intersection':
                    return "(" + self.node2string(left) + " & " + self.node2string(right) + ")"
                elif node.data == 'union':
                    return "(" + self.node2string(left) + " |" + self.node2string(right) + ")"
                elif node.data == 'difference':
                    return "(" + self.node2string(left) + " - " + self.node2string(right) + ")"
                elif node.data == 'symmetric_difference':
                    return "(" + self.node2string(left) + " ^ " + self.node2string(right) + ")"
                elif node.data == 'multiplication':
                    return "(" + self.node2string(left) + " * " + self.node2string(right) + ")"
                elif node.data == 'composition':
                    return "(" + self.node2string(left) + " ° " + self.node2string(right) + ")"
                elif node.data == 'is' or node.data == 'is_not':
                    if right.data == 'lattice':
                        return f"{self.node2string(left)} is lattice" if node.data == 'is' else f"{self.node2string(left)} is not lattice"
                    elif right.data == 'total_order':
                        return f"{self.node2string(left)} is total_order" if node.data == 'is' else f"{self.node2string(left)} is not total_order"
                    elif right.data == 'reflexive':
                        return f"{self.node2string(left)} is reflexive" if node.data == 'is' else f"{self.node2string(left)} is not reflexive"
                    elif right.data == 'symmetric':
                        return f"{self.node2string(left)} is symmetric" if node.data == 'is' else f"{self.node2string(left)} is not symmetric"
                    elif right.data == 'asymmetric':
                        return f"{self.node2string(left)} is asymmetric" if node.data == 'is' else f"{self.node2string(left)} is not asymmetric"
                    elif right.data == 'antisymmetric':
                        return f"{self.node2string(left)} is antisymmetric" if node.data == 'is' else f"{self.node2string(left)} is not antisymmetric"
                    elif right.data == 'transitive':
                        return f"{self.node2string(left)} is transitive" if node.data == 'is' else f"{self.node2string(left)} is not transitive"
                else:
                    raise Exception(f"ERROR! {node.data}")
            else:
                pass
                #raise Exception(f"ERROR! Unexpected input! {node}")
        elif isinstance(node, Token):
            return f"{node.value}"
        else:
            raise Exception(f"ERROR! unkonown instance {type(node)}")
            
            
    def explain(self, var, data):
    
        relation = self.get_variables()[var]

        if data == "lattice":
            yield from self.why_lattice(relation, var)
        elif data == "total_order":
            yield from self.why_total_order(relation, var)
        elif data == "reflexive":
            yield from self.why_reflexive(relation, var)
        elif data == "symmetric":
            yield from self.why_symmetric(relation, var)
        elif data == "asymmetric":
            yield from self.why_asymmetric(relation, var)
        elif data == "antisymmetric":
            yield from self.why_antisymmetric(relation, var)
        elif data == "transitive":
            yield from self.why_transitive(relation, var)
            
    def why_lattice(self, relation, name=None):
        if relation.is_lattice():
            yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is lattice! [domain: {relation.get_domain()}]"
        else:
            for x,y in relation.cartesian_product():
                if relation.infimum([x,y]) is None:
                    yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is not lattice because "+\
                    f"infimum [{x},{y}] is None!"
                if relation.supremum([x,y]) is None:
                    yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is not lattice because "+\
                    f"supremum [{x},{y}] is None!"
        
            
    def why_total_order(self, relation, name=None):
        if relation.is_total_order():
            yield f"<<EXPLAIN>> Relation {name}={self.get_relation()} is total_order! [domain: {self.get_domain()}]"
        else:
            for x in relation.get_domain():
                for y in relation.get_domain():
                    if not((x,y) in relation.get_relation() or (y,x) in relation.get_relation()):
                        yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is not total order because "+\
                        f"both elements ({x},{y}) and ({y},{x}) are missing! [domain: {relation.get_domain()}]"
        
            
    def why_reflexive(self, relation, name=None):
        if relation.is_reflexive():
            yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is reflexive! [domain: {relation.get_domain()}]"
        else:
            for x in relation.get_domain():
                if (x,x) not in relation.get_relation():
                    yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is not reflexive because "+\
                    f"element ({x},{x}) is missing ! [domain: {relation.get_domain()}]"
        
    
    def why_symmetric(self, relation, name=None):
        if relation.is_symmetric():
            yield f"<<EXPLAIN>> Relation {name}={self.get_relation()} is symmetric! [domain: {relation.get_domain()}]"
        else:
            for x,y in relation.get_relation():
                if (y,x) not in relation.get_relation():
                    yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is not symmetric because "+\
                    f"it contains element ({x},{y}), but element ({y},{x}) is missing! [domain: {relation.get_domain()}]"
        
            
    def why_asymmetric(self, relation, name=None):
        if relation.is_asymmetric():
            yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is asymmetric! [domain: {relation.get_domain()}]"
        else:
            for x,y in relation.get_relation():
                if (y,x) in relation.get_relation():
                    yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()}is not asymmetric because "+\
                    f"it contains both elements ({x},{y}) and ({y},{x})! [domain: {relation.get_domain()}]"

            
    def why_antisymmetric(self, relation, name=None):
        if relation.is_antisymmetric():
            yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is antisymmetric! [domain: {relation.get_domain()}]"
        else:
            for x,y in relation.get_relation():
                if (y,x) in relation.get_relation() and (x != y):
                    yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()}is not antisymmetric because "+\
                    f"it contains both elements ({x},{y}) and ({y},{x})! [domain: {relation.get_domain()}]"
        
            
    def why_transitive(self, relation, name=None):
        if relation.is_transitive() == True:
            yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is transitive! [domain: {relation.get_domain()}]"
        else:
            for x in relation.get_domain():
                for y in relation.is_out_relation_with(x):
                    for z in relation.is_out_relation_with(y):
                        if (x,z) not in relation.get_relation():
                            yield f"<<EXPLAIN>> Relation {name}={relation.get_relation()} is not transitive because "+\
                            f"it contains elements ({x},{y}) and ({y},{z}), but element ({x},{z}) is missing! "+\
                            f"[domain: {relation.get_domain()}]"
        

    def analyze(self, node, level=0, expected=True):

        actual = self.interpret(node)

        if actual != expected:

            yield f"<<<ERROR>>> {self.node2string(node)}"

            if isinstance(node, Tree):
                if len(node.children) <= 2:
                    if node.data == "negation":
                        self.analyze(node.children[0], level+1, not expected)
                    elif node.data == "conjunction":
                        if expected == True:
                            yield from self.analyze(node.children[0], level+1, True)
                            yield from self.analyze(node.children[1], level+1, True)
                        else:
                            if self.interpret(node.children[0]) == True:
                                yield from self.analyze(node.children[0], level+1, False)
                            if self.interpret(node.children[1]) == True:
                                yield from self.analyze(node.children[1], level+1, False)
                    elif node.data == "disjunction":
                        if expected == False:
                            yield from self.analyze(node.children[0], level+1, False)
                            yield from self.analyze(node.children[1], level+1, False)
                        else:
                            if self.interpret(node.children[0]) == False:
                                yield from self.analyze(node.children[0], level+1, True)
                            if self.interpret(node.children[1]) == False:
                                yield from self.analyze(node.children[1], level+1, True)
                    elif node.data == "implication":
                        if expected == False:
                            yield from self.analyze(node.children[0], level+1, True)
                            yield from self.analyze(node.children[1], level+1, False)
                        else:
                            if self.interpret(node.children[0]) == True:
                                yield from self.analyze(node.children[0], level+1, False)
                            if self.interpret(node.children[1]) == False:
                                yield from self.analyze(node.children[1], level+1, True)
                    elif node.data == 'is' or node.data == 'is_not':
                        yield from self.explain(node.children[0].value, node.children[1].data)
                             
            
    def analyze_all(self, node):
    
        if isinstance(node, Tree):
            if node.data == "implication":
                condition, conclusion = node.children
                yield from self.analyze(condition,0,True)
                yield from self.analyze(conclusion,0,False)
            else:
                raise Exception(f"ERROR! Input is NOT implication! {node}")
        else:
            raise Exception(f"ERROR! Unexpected input! {node}")

In [41]:
import unittest

class TestNotebook_interpreter(unittest.TestCase):

    relation1 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
    relation2 = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)
    relation3 = HomogeneousRelation({(1,1),(2,2),(3,3),(4,4)},{1,2,3,4}, True)
    universe1 = {1,2,3}
    
    def test_valid_interpreter(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertEqual(interpreter.get_variables(), variables)
        self.assertEqual(interpreter.get_universe(), self.universe1)
        
    def test_set_universe_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_universe = {1,2,3,4,'h','ch'}
        interpreter.set_universe(new_universe, True)
        self.assertEqual(interpreter.get_universe(), new_universe)
        
    def test_set_universe_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_universe = {2,3,4,'h','ch'}
        self.assertRaises(Exception, interpreter.set_universe,new_universe, True)
        
    def test_set_variables_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1, 'T':self.relation2}
        interpreter.set_variables(new_variables, True)
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_set_variables_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1, 'T':self.relation3}
        self.assertRaises(Exception, interpreter.set_variables,new_variables, True)
        
    def test_set_variable_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        interpreter.set_variable('S',self.relation2, True)
        new_variables =  {'R':self.relation1, 'S':self.relation2}
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_set_variable_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.set_variable,'T',self.relation2, True)
        
    def test_set_variable_not_valid_universe(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.set_variable,'R',self.relation3, True)
        
    def test_add_variable_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        interpreter.add_variable('T',self.relation2, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1,'T':self.relation2}
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_add_variable_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.add_variable,'R',self.relation2, True)
        
    def test_add_variable_not_valid_universe(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.add_variable,'T',self.relation3, True)
        
    def test_interpret_formula_01(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        tree = interpreter.parse("R is transitive and S is transitive and R | S is not transitive")
        self.assertEqual(interpreter.interpret(tree), False)
        
    def test_interpret_formula_02(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        tree = interpreter.parse("R is transitive and S is transitive -> R | S is transitive")
        self.assertEqual(interpreter.interpret(tree), True)
        
    def test_interpret_formula_03(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)       
        U = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
        V = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)
        interpreter.add_variable('U',U, True)
        interpreter.add_variable('V',V, True)       
        tree = interpreter.parse("V is lattice and U|V is transitive ->  V is total_order")
        self.assertEqual(interpreter.interpret(tree), True)
        
    def test_node2string_01(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "R is transitive and S is transitive -> R | S is transitive"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_02(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "R is reflexive or S is not transitive -> R ^ S is not asymmetric"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_03(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "R is antisymmetric and S is transitive -> R - S is lattice"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_04(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "S is reflexive and not(R is antisymmetric and S is transitive) -> R ° S is not transitive"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_05(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "(S is reflexive and R is not reflexive) and not(R is antisymmetric and S is transitive) -> R <= S"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_06(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "(S is reflexive and R is not reflexive) and R is reflexive"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_07(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "S is reflexive and (R is not reflexive and R is reflexive)"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
    
        
unittest.main(argv=[''], verbosity=1, exit=False)

................................................................................................................
----------------------------------------------------------------------
Ran 112 tests in 1.892s

OK


In [48]:
import unittest

class TestNotebook_explanator(unittest.TestCase):

    relation1 = HomogeneousRelation({(1,1),(2,2),(3,3),(1,2),(2,3)}, {1,2,3}, True)
    relation2 = HomogeneousRelation({(1,2),(2,1)}, {1,2,3}, True)
    variables = {'A':relation1,
                 'B':relation1,
                 'C':relation1,
                 'D':relation1,
                 'E':relation1,
                 'F':relation2}
    interpreter = Interpreter(variables, {1,2,3}, True)

    condition1 = interpreter.parse("A is symmetric and F is reflexive")
    condition2 = interpreter.parse("A is not reflexive and (B is not reflexive or C is not reflexive)")
    condition3 = interpreter.parse("(A is not reflexive and B is reflexive) -> (C is not reflexive and D is not reflexive)")
    condition4 = interpreter.parse("(A is not reflexive and (B is not reflexive or C is not reflexive)) -> D is not reflexive")
    condition5 = interpreter.parse("A is  reflexive -> B is reflexive")
    condition6 = interpreter.parse("A is  reflexive -> (B is reflexive or C is not reflexive)")
    condition7 = interpreter.parse("A is  reflexive -> (B is reflexive and C is reflexive)")
    
    def test_explanator_01(self):
        generator = self.interpreter.analyze_all(self.condition4)
        counter = 0
        for p in generator:
            counter += 1
        self.assertEqual(counter, 8)
        
    def test_explanator_02(self):
        generator = self.interpreter.analyze(self.condition1)
        counter = 0
        for p in generator:
            counter += 1
        self.assertEqual(counter, 8)

unittest.main(argv=[''], verbosity=1, exit=False)

.................................................................................................................
----------------------------------------------------------------------
Ran 113 tests in 1.739s

OK


In [33]:
#relation1 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
#relation2 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)

#interpreter = Interpreter({'R':relation1,'S':relation2}, {1,2,3}, True)
# #interpreter.set_variables({'R':relation1,'X':HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3,4}, True)}, True) #Error

In [34]:
#tree = interpreter.parse("R is transitive and S is transitive -> R | S is transitive")

#print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
#print(f"[RESULT -> {interpreter.interpret(tree)}]")

In [35]:
# tree = interpreter.parser.parse("R is transitive and S is transitive and not R | S is transitive")

# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

In [36]:
# U = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
# V = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)

# interpreter.add_variable('U',U, True)
# interpreter.add_variable('V',V, True)
# tree = interpreter.parser.parse("V is lattice and U|V is transitive ->  V is total_order")

# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

# print(f"\nvariables:\n {interpreter.get_variables()}")

In [37]:
# V = PartialOrder({(1,1),(2,2),(3,3)}, {1,2,3},True)

# #interpreter.add_variable('V',V) #Error - already defined variable
# interpreter.set_variable('V',V,True)
# #interpreter.set_universe({1,2},True) #Error - not valid universe

# tree = interpreter.parser.parse("V != V ->  S is total_order")




# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

# print(f"\nvariables:\n {interpreter.get_variables('V','S')}")

In [38]:
#interpreter.get_variables()

In [39]:
#interpreter.get_universe()